In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import pymysql
from collections import Counter

In [2]:
conn = pymysql.connect(host='chamaeleon.konkuk.ac.kr',
                       port = 3306,
                       user = "class_da",
                       password = "dataanalytics",
                       db = "tp_class_da")

cursor_ci = conn.cursor()
cursor_ci.execute("SELECT ci.특허등록번호 as pt_num, ci.인용특허등록연도 as ci_year FROM citation ci")
db_ci = cursor_ci.fetchall()

In [3]:
ci_year_list = []

for pt_num, ci_year in db_ci:
    ci_year_list.append((pt_num, ci_year))
    
ci_dict = {}
for pt_num, ci_year in ci_year_list:
    if ci_dict.get(pt_num):
        ci_dict[pt_num].append(ci_year)
    else :
        ci_dict[pt_num]=[ci_year]

In [4]:
#AI num
cursor_ai = conn.cursor()
cursor_ai.execute("SELECT ab.특허등록번호 AS pt_num, ab.특허초록텍스트 AS ai_abs FROM tp_class_da.abstract AS ab WHERE ab.특허초록텍스트 LIKE '%AI%'")
db_ai = cursor_ai.fetchall()

In [5]:
#AI abs
Ai_abs_list = []

for pt_num, ai_abs in db_ai:
    Ai_abs_list.append((pt_num, ai_abs))
    
Ai_dict = {}
for pt_num, ai_abs in Ai_abs_list:
    Ai_dict[pt_num]=[ai_abs]

In [6]:
Ai_ci_dict = {key:value for key, value in ci_dict.items() if key in Ai_dict.keys()}

In [7]:
for j in Ai_ci_dict.keys():
    new_lst = []
    for i in Ai_ci_dict[j]:
        new_lst.append(int(str(i)[:3] + '0'))
    Ai_ci_dict[j] = new_lst

In [8]:
Ai_ci_dict

{'9532496': [2000, 2010, 2010, 2000, 2010, 2010],
 '9532500': [1990,
  1930,
  2010,
  1960,
  1960,
  1960,
  1960,
  1970,
  1970,
  1990,
  1990,
  2000,
  2010],
 '9532501': [1910,
  1920,
  1940,
  1940,
  1940,
  1950,
  1960,
  1970,
  1970,
  1970,
  1970,
  1970,
  1970,
  1980,
  1980,
  1980,
  1990,
  1990,
  1990],
 '9532503': [2000,
  2010,
  2010,
  2010,
  2010,
  1980,
  1980,
  1980,
  1980,
  1980,
  1990,
  1990,
  1990,
  2000,
  2000,
  2000,
  2000,
  2010,
  2010,
  2000],
 '9532504': [2010, 2010, 1980, 1990, 1980, 1980, 1980, 2000, 2000, 2010],
 '9532507': [1960, 1950, 1960, 1970, 1970, 1990, 1950, 1960, 2010],
 '9532509': [1980,
  1990,
  2000,
  1990,
  2000,
  2010,
  2000,
  2000,
  2000,
  2000,
  1940,
  1950,
  1950,
  1950,
  1980,
  1990,
  1990,
  1980,
  1990,
  2000,
  2000,
  2000,
  2010,
  2010],
 '9532511': [1990,
  2010,
  2010,
  1910,
  1920,
  2000,
  1930,
  1930,
  2000,
  2000,
  2010,
  2010,
  2010,
  2010,
  1940,
  1850,
  1940,
  194

In [13]:
year_list = [1850, 1860, 1870, 1880, 1890, 1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010]

df = DataFrame(columns = year_list ,index = Ai_ci_dict.keys())

In [14]:
for i in Ai_ci_dict.keys():
    for j in year_list:
        df.at[i, j] = Counter(Ai_ci_dict[i])[j]
df
df.to_csv("patent_ci_year.csv")

In [15]:
df

,1850,1860,1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010
9532496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4
9532500,0,0,0,0,0,0,0,0,1,0,0,4,2,0,3,1,2
9532501,0,0,0,0,0,0,1,1,0,3,1,1,6,3,3,0,0
9532503,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,6,6
9532504,0,0,0,0,0,0,0,0,0,0,0,0,0,4,1,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9854706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,2
9854707,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,5
9854710,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,21
9854711,0,0,0,0,1,0,1,3,3,1,4,9,6,4,12,27,10
